In [1]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.8 MB/s 


# 첫 번째 과정 - 모든 유저들 끼리의 유사도을 구성한다.

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import pearsonr
import pymysql

connection = pymysql.connect(
    user="j7e103",
    passwd="000000",
    host="j7e103.p.ssafy.io",
    port=3307,
    db="homidu",
    charset="utf8",
)
cursor = connection.cursor(pymysql.cursors.DictCursor)

In [ ]:
SQL = 'SELECT * FROM plantLike2'
plant_like = pd.read_sql(SQL, connection)

In [ ]:
plant_like

,plant_id,user_id,age,score
0,1,1,None,4
1,1,2,None,1
2,1,3,None,4
3,1,4,None,2
4,1,5,None,0
5,1,6,None,0
6,1,7,None,1
7,1,8,None,0
8,1,9,None,4
9,2,1,None,0


In [ ]:
# dataframe 세팅 index = user_id, columns = plant_id
zeros = np.zeros((len(set(plant_like['user_id'])), len(set(plant_like['plant_id']))))
farm = pd.DataFrame(zeros, index=list(set(plant_like['user_id'])), columns=sorted(list(set(plant_like['plant_id']))))
farm

,1,2,3,4
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


In [ ]:
# 선호도 값 채우기
for uid, pid, score in zip(plant_like['user_id'], plant_like['plant_id'], plant_like['score']):
    farm[pid].loc[uid] = score
farm

,1,2,3,4
1,4.0,0.0,3.0,2.0
2,1.0,0.0,0.0,0.0
3,4.0,0.0,1.0,0.0
4,2.0,3.0,3.0,4.0
5,0.0,2.0,0.0,3.0
6,0.0,0.0,1.0,0.0
7,1.0,1.0,2.0,4.0
8,0.0,0.0,1.0,3.0
9,4.0,0.0,0.0,1.0


In [ ]:
# 유사도 테이블 생성
n = len(farm.index)
zeros = np.zeros((n, n))
sim = pd.DataFrame(zeros, index=list(farm.index), columns=list(farm.index))
sim

,1,2,3,4,5,6,7,8,9
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 유사도 계산
for name1 in sim.index:
    for name2 in sim.index:
        sim[name1].loc[name2] = round(pearsonr(list(farm.loc[name1]), list(farm.loc[name2]))[0], 4) # scipy 라이브러리 사용
sim

,1,2,3,4,5,6,7,8,9
1,1.0000,0.6831,0.7991,-0.4781,-0.6831,0.2928,0.0000,0.0000,0.6960
2,0.6831,1.0000,0.9685,-0.8165,-0.5556,-0.3333,-0.4714,-0.4714,0.9685
3,0.7991,0.9685,1.0000,-0.8627,-0.7337,-0.0880,-0.4981,-0.4981,0.9070
4,-0.4781,-0.8165,-0.8627,1.0000,0.8165,0.0000,0.8660,0.8660,-0.6470
5,-0.6831,-0.5556,-0.7337,0.8165,1.0000,-0.5556,0.6285,0.6285,-0.3815
6,0.2928,-0.3333,-0.0880,0.0000,-0.5556,1.0000,0.0000,0.0000,-0.4402
7,0.0000,-0.4714,-0.4981,0.8660,0.6285,0.0000,1.0000,1.0000,-0.2490
8,0.0000,-0.4714,-0.4981,0.8660,0.6285,0.0000,1.0000,1.0000,-0.2490
9,0.6960,0.9685,0.9070,-0.6470,-0.3815,-0.4402,-0.2490,-0.2490,1.0000


In [ ]:
# 모든 유저간의 유사도 행렬 구성 완료 (첫 번째 과정)
sim.to_csv('sim.csv')

# 두 번째 과정 - 업데이트된 사람만 유사도 행렬을 업데이트 해준다.

In [ ]:
connection = pymysql.connect(
    user="j7e103",
    passwd="000000",
    host="j7e103.p.ssafy.io",
    port=3307,
    db="homidu",
    charset="utf8",
)
cursor = connection.cursor(pymysql.cursors.DictCursor)

In [ ]:
SQL = 'SELECT * FROM plantLike2'
plant_like = pd.read_sql(SQL, connection)

In [ ]:
plant_like

,plant_id,user_id,age,score
0,1,1,None,4
1,1,2,None,1
2,1,3,None,4
3,1,4,None,2
4,1,5,None,0
5,1,6,None,0
6,1,7,None,1
7,1,8,None,0
8,1,9,None,4
9,2,1,None,0


In [ ]:
zeros = np.zeros((len(set(plant_like['user_id'])), len(set(plant_like['plant_id']))))
farm = pd.DataFrame(zeros, index=list(set(plant_like['user_id'])), columns=sorted(list(set(plant_like['plant_id']))))

In [ ]:
farm

,1,2,3,4
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


In [ ]:
for uid, pid, score in zip(plant_like['user_id'], plant_like['plant_id'], plant_like['score']):
    farm[pid].loc[uid] = score

In [ ]:
farm

,1,2,3,4
1,4.0,0.0,3.0,2.0
2,1.0,0.0,0.0,0.0
3,4.0,0.0,1.0,0.0
4,2.0,3.0,3.0,4.0
5,0.0,2.0,0.0,3.0
6,0.0,0.0,1.0,0.0
7,1.0,1.0,2.0,4.0
8,0.0,0.0,1.0,3.0
9,4.0,0.0,0.0,1.0


In [ ]:
sim = pd.read_csv('sim.csv', index_col=0)
sim.columns = sim.index
sim

,1,2,3,4,5,6,7
1,1.0000,-0.8704,-0.7817,0.8528,0.4062,0.5222,0.7385
2,-0.8704,1.0000,0.9685,-0.8165,-0.5556,-0.3333,-0.4714
3,-0.7817,0.9685,1.0000,-0.8627,-0.7337,-0.0880,-0.4981
4,0.8528,-0.8165,-0.8627,1.0000,0.8165,0.0000,0.8660
5,0.4062,-0.5556,-0.7337,0.8165,1.0000,-0.5556,0.6285
6,0.5222,-0.3333,-0.0880,0.0000,-0.5556,1.0000,0.0000
7,0.7385,-0.4714,-0.4981,0.8660,0.6285,0.0000,1.0000


In [ ]:
# 1번이 변경 됐고, 8번 9번은 신규로 들어옴
update_arr = [1, 8, 9]

In [ ]:
for update_uid in update_arr:

    # sim.index <- sim행렬에 존재하는 모든 사람의 리스트    
    
    # update_uid가 기존의 유사도 행렬에 없는 유저의 경우
    # 값을 1로 초기화 해줌
    if update_uid not in sim.index:
        sim[update_uid] = 1
        sim.loc[update_uid] = 1


    # update_uid와 i의 피어슨 유사도 계산
    for i in sim.index:
        # 피어슨 유사도가 2번 계산 되던걸 한번으로 줄임
        pearson = round(pearsonr(list(farm.loc[update_uid]), list(farm.loc[i]))[0], 4)
        sim[update_uid].loc[i] = pearson
        sim[i].loc[update_uid] = pearson

In [ ]:
sim

,1,2,3,4,5,6,7,8,9
1,1.0000,0.6831,0.7991,-0.4781,-0.6831,0.2928,0.0000,0.0000,0.6960
2,0.6831,1.0000,0.9685,-0.8165,-0.5556,-0.3333,-0.4714,-0.4714,0.9685
3,0.7991,0.9685,1.0000,-0.8627,-0.7337,-0.0880,-0.4981,-0.4981,0.9070
4,-0.4781,-0.8165,-0.8627,1.0000,0.8165,0.0000,0.8660,0.8660,-0.6470
5,-0.6831,-0.5556,-0.7337,0.8165,1.0000,-0.5556,0.6285,0.6285,-0.3815
6,0.2928,-0.3333,-0.0880,0.0000,-0.5556,1.0000,0.0000,0.0000,-0.4402
7,0.0000,-0.4714,-0.4981,0.8660,0.6285,0.0000,1.0000,1.0000,-0.2490
8,0.0000,-0.4714,-0.4981,0.8660,0.6285,0.0000,1.0000,1.0000,-0.2490
9,0.6960,0.9685,0.9070,-0.6470,-0.3815,-0.4402,-0.2490,-0.2490,1.0000


In [ ]:
# 업데이트된 사람들만 유사도행렬을 새로 계산해줌
sim.to_csv('sim.csv')

In [ ]:
sim = pd.read_csv('sim.csv', index_col=0)
sim.columns = sim.index
sim

,1,2,3,4,5,6,7,8,9
1,1.0000,0.6831,0.7991,-0.4781,-0.6831,0.2928,0.0000,0.0000,0.6960
2,0.6831,1.0000,0.9685,-0.8165,-0.5556,-0.3333,-0.4714,-0.4714,0.9685
3,0.7991,0.9685,1.0000,-0.8627,-0.7337,-0.0880,-0.4981,-0.4981,0.9070
4,-0.4781,-0.8165,-0.8627,1.0000,0.8165,0.0000,0.8660,0.8660,-0.6470
5,-0.6831,-0.5556,-0.7337,0.8165,1.0000,-0.5556,0.6285,0.6285,-0.3815
6,0.2928,-0.3333,-0.0880,0.0000,-0.5556,1.0000,0.0000,0.0000,-0.4402
7,0.0000,-0.4714,-0.4981,0.8660,0.6285,0.0000,1.0000,1.0000,-0.2490
8,0.0000,-0.4714,-0.4981,0.8660,0.6285,0.0000,1.0000,1.0000,-0.2490
9,0.6960,0.9685,0.9070,-0.6470,-0.3815,-0.4402,-0.2490,-0.2490,1.0000


# get_recommendation 함수 사용

In [ ]:
sim = pd.read_csv('sim.csv', index_col=0)
sim.columns = sim.index
sim

,1,2,3,4,5,6,7,8,9
1,1.0000,0.6831,0.7991,-0.4781,-0.6831,0.2928,0.0000,0.0000,0.6960
2,0.6831,1.0000,0.9685,-0.8165,-0.5556,-0.3333,-0.4714,-0.4714,0.9685
3,0.7991,0.9685,1.0000,-0.8627,-0.7337,-0.0880,-0.4981,-0.4981,0.9070
4,-0.4781,-0.8165,-0.8627,1.0000,0.8165,0.0000,0.8660,0.8660,-0.6470
5,-0.6831,-0.5556,-0.7337,0.8165,1.0000,-0.5556,0.6285,0.6285,-0.3815
6,0.2928,-0.3333,-0.0880,0.0000,-0.5556,1.0000,0.0000,0.0000,-0.4402
7,0.0000,-0.4714,-0.4981,0.8660,0.6285,0.0000,1.0000,1.0000,-0.2490
8,0.0000,-0.4714,-0.4981,0.8660,0.6285,0.0000,1.0000,1.0000,-0.2490
9,0.6960,0.9685,0.9070,-0.6470,-0.3815,-0.4402,-0.2490,-0.2490,1.0000


In [ ]:
# plant_like 가져오는 코드 생략
plant_like

,plant_id,user_id,age,score
0,1,1,None,4
1,1,2,None,1
2,1,3,None,4
3,1,4,None,2
4,1,5,None,0
5,1,6,None,0
6,1,7,None,1
7,1,8,None,0
8,1,9,None,4
9,2,1,None,0


plant_like가 200만행이고, 읽어오는데 30초가 걸림 (코랩 기준)
즉 유사도가 계산되어 있다 하더라도, plant_like 테이블을 읽어오는데 30초가 넘게 걸리니 사용자는 추천된 결과를 보려면 3~40초는 기다려야 한다는 뜻임
(유사도와 별개로 선호도 테이블을 전부 가져와야 하니까)

이 부분을 개선하려면 선호도를 가져오는 방법을 개선해야함
테이블의 구조를 바꾸던가..

In [ ]:
def get_recommendation(uid):
    plants = list(farm.columns)
    score_dic = defaultdict(int)  # 평점
    sim_dic = defaultdict(int)  # 유사도

    for uid2, sim_score in dict(sim[uid]).items():
        if sim_score > 0 and uid != uid2:
            for plant in plants:
                score = 0
                if not farm.loc[uid][plant]:
                    score += farm.loc[uid2][plant] * sim_score
                    score_dic[plant] += score
                    sim_dic[plant] += sim_score

    ret = []
    for key in score_dic.keys():
        score_dic[key] = score_dic[key] / sim_dic[key]
        ret.append([score_dic[key], key])

    return sorted(ret, key=lambda x: x[0], reverse=True)

get_recommendation(2)

[[1.1517880996908514, 3], [0.8910728598145109, 4], [0.0, 2]]

In [ ]:
# 상위 10명을 추천하는 것 보다, 나와의 유사도가 상위 10%인 사람들만 계산해서 추천해주는게 더 합리적일 것 같아서
# 퍼센트로 바꿈

def get_recommendation_top_percent(uid, percent):
    top_cnt = int(len(sim.index) * ((100 - percent) / 100))
    plants = list(farm.columns)
    score_dic = defaultdict(int)  # 평점
    sim_dic = defaultdict(int)  # 유사도
    for uid2, sim_score in sorted(dict(sim[2]).items(), key=lambda x: x[1], reverse=True)[1:top_cnt + 1]:
        if sim_score > 0 and uid != uid2:  # 박현정과 유사도가 0 이상인 사람에 대해서
            for plant in plants:  # 모든 식물을 조사해본다.
                score = 0
                if not farm.loc[uid][plant]:  # 만약 박현정이 그 식물을 키우지 않는다면,
                    score += farm.loc[uid2][plant] * sim_score  # 평점 예측 
                    score_dic[plant] += score
                    sim_dic[plant] += sim_score

    # print(score_dic)
    # print(sim_dic)
    ret = []
    for key in score_dic.keys():
        score_dic[key] = score_dic[key] / sim_dic[key]
        ret.append([score_dic[key], key])

    return sorted(ret, key=lambda x: x[0], reverse=True)

get_recommendation_top_percent(2, 50)

[[1.1517880996908516, 3], [0.891072859814511, 4], [0.0, 2]]

# 0929 - 문자열로 인코딩된 선호도 테이블 파싱

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import pearsonr
import pymysql

connection = pymysql.connect(
    user="j7e103",
    passwd="000000",
    host="j7e103.p.ssafy.io",
    port=3307,
    db="homidu",
    charset="utf8",
)
cursor = connection.cursor(pymysql.cursors.DictCursor)

In [5]:
SQL = 'SELECT * FROM plant_like'
plant_like = pd.read_sql(SQL, connection)

In [6]:
plant_like

,user_id,score
0,1,0100000000000000000000000000000000000000000000...
1,2,0100000000000000000000000000000000000000000000...
2,3,0100000000000000000000000000000000000000000000...
3,4,0100000000000000000000000000000000000000000000...
4,5,0100000000000000000000000000000000000000000000...
5,6,0100000000000000000000000000000000000000000000...
6,7,0100000000000000000000000000000000000000000000...
7,8,0100000000000000000000000000000000000000000000...
8,9,0100000000000000000000000000000000000000000000...
9,10,0100000000000000000000000000000000000000000000...


In [13]:
len(plant_like['user_id'])

12

In [14]:
len(plant_like['score'].iloc[0])

215

In [17]:
zeros = np.zeros((len(set(plant_like['user_id'])), len(plant_like['score'].iloc[0])))
farm = pd.DataFrame(zeros, index=list(set(plant_like['user_id'])), columns=range(1, len(plant_like['score'].iloc[0]) + 1))

In [18]:
farm

,1,2,3,4,5,6,7,8,9,10,...,206,207,208,209,210,211,212,213,214,215
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
plant_like

,user_id,score
0,1,0100000000000000000000000000000000000000000000...
1,2,0100000000000000000000000000000000000000000000...
2,3,0100000000000000000000000000000000000000000000...
3,4,0100000000000000000000000000000000000000000000...
4,5,0100000000000000000000000000000000000000000000...
5,6,0100000000000000000000000000000000000000000000...
6,7,0100000000000000000000000000000000000000000000...
7,8,0100000000000000000000000000000000000000000000...
8,9,0100000000000000000000000000000000000000000000...
9,10,0100000000000000000000000000000000000000000000...


In [1]:
for uid, score in zip(plant_like['user_id'], plant_like['score']):
    for i, val in enumerate(score):
        farm[i + 1].loc[uid] = val

NameError: name 'plant_like' is not defined

In [ ]:
farm

,1,2,3,4,5,6,7,8,9,10,...,206,207,208,209,210,211,212,213,214,215
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,4,0
6,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
SQL = 'SELECT * FROM update_table'
update_table = pd.read_sql(SQL, connection)

In [27]:
update_table

,user_id
0,1
1,8
2,9


In [30]:
update_arr = list(update_table['user_id'])

In [31]:
update_arr

[1, 8, 9]